In [28]:
import pandas as pd

# Excel fájl beolvasása xlsx formatumban
# pandas Excel-fájlok beolvasásához (pl. .xlsx) az openpyxl nevű modult használja ---> TELEPITENI KELL
df = pd.read_excel("../data/inventoryAI_data/leltar.XLSX")

# Első 5 sor megjelenítése
df.head()
print("---------------------------------------------------------")
# Oszlopnevek listája
print("Oszlopnevek:")
print(df.columns)
print("---------------------------------------------------------")
# Adattípusok és hiányzó értékek
print("Adattipusok és hiányzó értékek:")
print(df.info())
print("---------------------------------------------------------")
# Hiányzó értékek száma
print("Hiányzó értékek száma:")
print(df.isnull().sum())

---------------------------------------------------------
Oszlopnevek:
Index(['Eszköz', 'Alszám', 'Aktiválás dátuma', 'Eszköz megnevezése',
       'Besz.ért.', 'Kum. ÉCS', 'K.sz.ért', 'Pénznem', 'Mennyiség',
       'Leltárszám', 'Telephely'],
      dtype='object')
---------------------------------------------------------
Adattipusok és hiányzó értékek:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Eszköz              49 non-null     int64         
 1   Alszám              49 non-null     int64         
 2   Aktiválás dátuma    49 non-null     datetime64[ns]
 3   Eszköz megnevezése  49 non-null     object        
 4   Besz.ért.           49 non-null     int64         
 5   Kum. ÉCS            49 non-null     int64         
 6   K.sz.ért            49 non-null     int64         
 7   Pénznem             49 non-null  

➡️ Ez azt jelenti, hogy 49 sor (tehát 49 külön eszköz vagy tétel) található az adatban.

int64 → egész szám (pl. azonosítók, értékek, mennyiség)

datetime64[ns] → dátum formátum (itt az „Aktiválás dátuma”)

object → szöveg típus (pl. „Eszköz megnevezése”, „Telephely”, „Pénznem”)

Minden oszlopban 49 nem hiányzó érték van, tehát nincs hiányzó adat.

In [29]:
# Cél: az eszközneveket egységes formátumba hozni, kisbetűsre, whitespace javítás
df['Eszköz megnevezése'] = df['Eszköz megnevezése'].str.lower()  # kisbetűsítés
df['Eszköz megnevezése'] = df['Eszköz megnevezése'].str.strip()  # felesleges whitespace eltávolítása
df['Eszköz megnevezése'] = df['Eszköz megnevezése'].str.replace(r'\s+', ' ', regex=True)  # többszörös szóközök 1-re
# ---------------------------------------------------------
# 1. Duplikált sorok ellenőrzése
# ---------------------------------------------------------
duplikalt_sorok = df[df.duplicated()]
print("Duplikált sorok száma:", len(duplikalt_sorok))
if len(duplikalt_sorok) > 0:
    print("Duplikált sorok listája:")
    print(duplikalt_sorok)
else:
    print("Nincsenek duplikált sorok.")
print("---------------------------------------------------------")

# ---------------------------------------------------------
# 2. Nulla vagy negatív értékek vizsgálata a pénzügyi oszlopokban
# ---------------------------------------------------------
penzugyi_oszlopok = ['Besz.ért.', 'Kum. ÉCS', 'K.sz.ért']
for oszlop in penzugyi_oszlopok:
    nullak = (df[oszlop] == 0).sum()
    negativak = (df[oszlop] < 0).sum()
    print(f"{oszlop} - 0 értékek: {nullak}, negatív értékek: {negativak}")
print("---------------------------------------------------------")

# ---------------------------------------------------------
# 3. Egyedi értékek száma fontos mezőkben
# ---------------------------------------------------------
print("Egyedi eszköz megnevezések száma:", df['Eszköz megnevezése'].nunique())
print("Egyedi leltárszámok száma:", df['Leltárszám'].nunique())
print("Egyedi telephelyek száma:", df['Telephely'].nunique())
print("---------------------------------------------------------")

# ---------------------------------------------------------
# 4. Leggyakoribb telephelyek és eszközök
# ---------------------------------------------------------
print("Top 5 telephely gyakoriság szerint:")
print(df['Telephely'].value_counts().head())
print("---------------------------------------------------------")
print("Top 5 eszköz megnevezés gyakoriság szerint:")
print(df['Eszköz megnevezése'].value_counts().head())
print("---------------------------------------------------------")

# ---------------------------------------------------------
# 5. Dátumok ellenőrzése (nem üres, nem hibás)
# ---------------------------------------------------------
print("Aktiválás dátuma - minimum:", df['Aktiválás dátuma'].min())
print("Aktiválás dátuma - maximum:", df['Aktiválás dátuma'].max())
print("---------------------------------------------------------")

# ---------------------------------------------------------
# 6. Alap statisztika a numerikus oszlopokra
# ---------------------------------------------------------
print("Numerikus oszlopok statisztikai összefoglalója:")
print(df.describe())
print("---------------------------------------------------------")

# ---------------------------------------------------------
# 7. Kategorizálás / csoportosítás
# ---------------------------------------------
# Cél: hány eszköz van telephelyenként és az összes mennyiség
telephely_group = df.groupby('Telephely').agg(
    eszkoz_db=('Eszköz megnevezése', 'count'),
    mennyiseg_osszes=('Mennyiség', 'sum')
).reset_index()

print("Telephelyek összesítve (db és mennyiség):")
print(telephely_group)
print("---------------------------------------------------------")
# 8. Rövid összefoglaló
# ---------------------------------------------------------
print("ADAT-ELŐKÉSZÍTÉS ÖSSZEFOGLALÓJA")
print("--------------------------------")
print("Sorok száma:", len(df))
print("Oszlopok száma:", len(df.columns))
print("Hiányzó értékek összesen:", df.isnull().sum().sum())
print("Duplikált sorok:", len(duplikalt_sorok))
print("Telephelyek száma:", df['Telephely'].nunique())
print("Egyedi eszközök:", df['Eszköz megnevezése'].nunique())
print("--------------------------------")


Duplikált sorok száma: 0
Nincsenek duplikált sorok.
---------------------------------------------------------
Besz.ért. - 0 értékek: 49, negatív értékek: 0
Kum. ÉCS - 0 értékek: 49, negatív értékek: 0
K.sz.ért - 0 értékek: 49, negatív értékek: 0
---------------------------------------------------------
Egyedi eszköz megnevezések száma: 28
Egyedi leltárszámok száma: 49
Egyedi telephelyek száma: 11
---------------------------------------------------------
Top 5 telephely gyakoriság szerint:
Telephely
KGKMMI3330    22
BGKGYT1021    10
NIKSZF4421     4
GFBMI76630     3
BGKMJI1040     2
Name: count, dtype: int64
---------------------------------------------------------
Top 5 eszköz megnevezés gyakoriság szerint:
Eszköz megnevezése
doctus3.0knowledgebased system szoftver     11
doctus 3.0 knowledge based system szoftv    10
sw-ccme-ul-7940 cisco callmanager licenc     3
szoftver xilinx                              1
szoftver testxpert mesterprogram             1
Name: count, dtype: int64
---

# **🔹 Adatelőkészítés összefoglaló és megfigyelések**

**Adatminőség:**
Nincsenek hiányzó vagy duplikált sorok, így a sor-szintű adatok tiszták.

**Hiányzó vagy problémás adatok:**
A pénzügyi oszlopok (Besz.ért., Kum. ÉCS, K.sz.ért) mind 0 értékűek, az aktiválás dátuma pedig minden sorban azonos. Demo adatról van szó, ezért ezekből nem lehet értékelhető információt nyerni, és a chatbotban jelenleg nem relevánsak.

**Szöveges adatok tisztítása:**
Az eszközneveket kisbetűsítettük, whitespace-eket javítottuk, így egységesebbek. Ez lehetővé teszi, hogy a RAG modell pontosabban keressen az eszköznevekre.
Még vannak kisebb eltérések (pl. „doctus3.0knowledgebased” vs „doctus 3.0 knowledge based”), amit további manuális vagy szótár alapú javítással lehetne egységesíteni.

**Telephelyek és eszközök:**
11 telephely szerepel, a legtöbb eszköz a KGKMMI3330 telephelyen található (22 db).
Egyes eszközök többször is szerepelnek (pl. Doctus szoftver 11 + 10 alkalommal), de a legtöbb eszköz 1-3 példányban van.
Ez az információ hasznos a RAG modell tesztkérdéseihez, például: „Hol található a legtöbb eszköz?” → KGKMMI3330.

**Numerikus adatok:**
A Mennyiség mindenhol 1, a Leltárszám változatos → a leltárszám alkalmas lehet egyedi azonosításra a kérdés-válasz modellben.

## **Összegzés:**

Az adatok alapvetően tiszták és előkészítettek a további feldolgozásra. A szöveges oszlopok egységesítése megtörtént, így a RAG alapú chatbot hatékonyabban tud keresni és válaszolni az eszközökkel kapcsolatos kérdésekre. A demo jellegű pénzügyi és dátumadatok jelenleg nem relevánsak.